In [127]:
from __future__ import annotations
from typing import Type, Union
from copy import deepcopy

In [128]:
class Logic:
    def execute(self) -> None:
        pass

In [129]:
class Game:
    def __init__(self, player:Type[Player]) -> None:
        self.commands:list[dict[str, Type[Logic]]] = [] # str is the command to call, Type[Logic] is the logic to be executed.
        self.player:Type[Player] = player

    def set_player(self, player:Type[Player]) -> None:
        self.player = player

    def add_command(self, command:str, logic:Type[Logic]):
        self.commands.append({command:logic})

    def run(self) -> None:
        while True:
            self.describe()
            userInput = str(input("What would you like to do: ")).lower()

            match userInput:
                case "quit":
                    break
                case other:
                    self.process(userInput)

    def process(self, userInput:str) -> None:
        arguments = userInput.split()
        commandCaller = arguments[0]

        if commandCaller == "describe":
            self.describe()
            return None

        for i in self.commands:
            for _, (j, x) in enumerate(i.items):
                if commandCaller == j:
                    x.execute()
                    return None
        else:
            print("Command not found: {}".format(commandCaller))
            return None
    
    def describe(self) -> None:
        self.player.currentLocation.describe()

In [130]:
class Item:
    def __init__(self, id:int, name:str, amount:int, maxStack:int) -> None:
        self.id:int = id
        self.name:str = name
        self.amount:int = amount
        self.maxStack:int = maxStack

In [131]:
class Entity:
    def __init__(self, id:int, name:str) -> None:
        self.id:int = id
        self.name:str = name
    
    def behavior(self) -> None:
        pass

In [132]:
class Location:
    def __init__(self, id:int, name:str, description:str, north:Type[Location] = None, items:list[Type[Item]] = [], entities:list[Type[Entity]] = [], east:Type[Location] = None, south:Type[Location] = None, west:Type[Location] = None) -> None:
        self.id:int = id
        self.name:str = name
        self.items:list[Type[items]] = items
        self.entities:list[Type[Entity]] = entities
        self.description:str = description
        self.north:Type[Location] = north
        self.east:Type[Location] = east
        self.south:Type[Location] = south
        self.west:Type[Location] = west
    
    #

    def activate(self) -> None:
        pass

    def describe(self) -> None:
        currentEntities:dict[str, int] = {}
        for i in self.entities:
            if i.name in currentEntities:
                currentEntities[i.name] += 1
            else:
                currentEntities.update({i.name:1})
        
        currentItems:dict[str, int] = {}
        for i in self.items:
            if i.name in currentItems:
                currentItems[i.name] += 1
            else:
                currentItems.update({i.name:1})
        
        entitiesDescribe:str = ", ".join("{} {}".format(entityName, entityCount) for _, (entityName, entityCount) in enumerate(currentEntities.items()))
        itemsDescribe:str = ", ".join("{} {}".format(itemName, itemCount) for _, (itemName, itemCount) in enumerate(currentItems.items()))
        generalDescribe:str = self.description

        if currentEntities != {}:
            generalDescribe += ".\n{}".format(entitiesDescribe)
        if currentItems != {}:
            generalDescribe += ".\n{}".format(itemsDescribe)
        
        if self.entities != []:
            self.activate()

    #

    def change_north(self, location:Type[Location]) -> None:
        self.north = location
    
    def change_east(self, location:Type[Location]) -> None:
        self.east = location
    
    def change_south(self, location:Type[Location]) -> None:
        self.south = location
    
    def change_west(self, location:Type[Location]) -> None:
        self.west = location
    
    def change_location(self, direction:str, location:Type[Location]) -> None:
        direction = direction.lower()
        match direction:
            case "north":
                self.change_north(location)
            case "east":
                self.change_east(location)
            case "south":
                self.change_south(location)
            case "west":
                self.change_west(location)
    #

    def get_north(self) -> Type[Location]:
        return self.north
    
    def get_east(self) -> Type[Location]:
        return self.east
    
    def get_south(self) -> Type[Location]:
        return self.south
    
    def get_west(self) -> Type[Location]:
        return self.west

    def get_location(self, direction:str) -> Type[Location]:
        direction = direction.lower()
        match direction:
            case "north":
                return self.get_north()
            case "east":
                return self.get_east()
            case "south":
                return self.get_south()
            case "west":
                return self.get_west()

    def add_item_on_ground(self, item:Type[Item]) -> None:
        self.items.append(item)
    
    def add_items_on_gruond(self, items:list[Type[Item]]) -> None:
        self.items.extend(items)
    
    def add_entity_on_location(self, entity:Type[Entity]) -> None:
        self.entities.append(entity)
    
    def add_entities_on_location(self, entities:list[Type[Entity]]) -> None:
        self.entities.extend(entities)

In [133]:
class Player:
    def __init__(self, id:int, name:str, health:Union[float, int], currentLocation:Type[Location]) -> None:
        self.id:int = id
        self.name:str = name
        self.currentHealth:Union[float, int] = health
        self.maxHealth:Union[float, int] = health
        self.currentLocation:Type[Location] = currentLocation

    def difference_health(self, amount:Union[float, int]) -> None:
        if self.currentHealth - amount < 0:
            return 0
        return self.currentHealth - amount

    def check_max(self, amount:Union[float, int]) -> None:
        if self.currentHealth + amount > self.maxHealth:
            return self.maxHealth
        return self.currentHealth + amount

    def take_damage(self, amount:Union[float, int]) -> None:
        diff:Union[float, int] = self.difference_health(amount)
        self.currentHealth = diff

    def add_health(self, amount:Union[float, int]) -> None:
        checkMax:Union[float, int] = self.check_max(amount)
        self.currentHealth = checkMax

    def change_location(self, direction:str) -> None:
        direction = direction.lower()
        newLocation = self.currentLocation.get_location(direction=direction)
        directions = [self.currentLocation.north, self.currentLocation.east, self.currentLocation.south, self.currentLocation.west]
        for i in directions:
            if i != None:
                if newLocation.id == i.id:
                    self.currentLocation = i
                    print("You've gone to {}".format(self.currentLocation.name))
                    return None
                print("There's no such location.")
    
    def set_current_location(self, location:Type[Location]) -> None:
        self.currentLocation = location

In [134]:
class NPC(Entity):
    def __init__(self, id: int, name: str) -> None:
        super().__init__(id, name)
    
    def behavior(self) -> None:
        return super().behavior()
    
    def talk(self) -> None:
        pass

In [135]:
class Enemy(Entity):
    def __init__(self, id: int, name: str, health:Union[float, int]) -> None:
        self.currentHealth:Union[float, int] = health
        self.maxHealth:Union[float, int] = health
        super().__init__(id, name)
    
    def behavior(self) -> None:
        return super().behavior()

    # How the enemy will take damage.
    def take_damage(self) -> None:
        pass

    # How the enemy will attack.
    def attack(self) -> None:
        pass

In [136]:
class EnemyQuestion(Enemy):
    def __init__(self, id: int, name: str, health: Union[float, int]) -> None:
        super().__init__(id, name, health)
    
    def behavior(self) -> None:
        return super().behavior()
    
    def take_damage(self) -> None:
        return super().take_damage()
    
    # The attack should be prompting a question.
    def attack(self) -> None:
        return super().attack()

In [137]:
class World:
    pass

----------------------------------------------------

In [138]:
# firstLocation = Location(1, "test location 1", "")
# secondLocation = Location(2, "test location 2", "")
# firstLocation.change_north(secondLocation)
# secondLocation.change_south(firstLocation)
# player = Player(1, "test", 4, firstLocation)

In [139]:
# player.change_location("north")

In [140]:
# print(player.currentLocation.name)

In [141]:
# player.change_location("south")

------------------------------------

In [142]:
firstLocation = Location(1, "test location 1", "")
secondLocation = Location(2, "test location 2", "")
firstLocation.change_north(secondLocation)
secondLocation.change_south(firstLocation)
player = Player(1, "test", 4, firstLocation)

In [144]:
game = Game(player=player)
game.run()